![CoSAppLogo](images/cosapp.svg) **CoSApp** tutorials: Multi-points design

# Multi-points design

Up to now, systems have been solved in their up-and-running state, that is all parameters are set, and you want to obtain the state of the system for a given set of boundary conditions.

As CoSApp focuses on system design, we will now show how you can free parameters and compute them to satisfy design equations.
In complex systems, parameters are designed under most critical constraints, which will usually occur in different operating conditions.
As will be illustrated here, CoSApp is able to solve such multi-points design calculations.

## Case description

The simple circuit test case having a constant intensity source will be used here. In addition to the classical resolution driving the potential `V` at *node1* and *node2* to balance current fluxes, the two resistances have to be obtained from two different operating points.

![simple-circuit](images/design_circuit.svg)

| Operating point | Boundary conditions | Design variable | Design equation |
|---|---|---|---|
| Point 1 | $I_{source} = 0.08$ A | `R2.R` | `n2.V = 8 V` |
| Point 2 | $I_{source} = 0.15$ A | `R1.R` | `n1.V = 50 V` |

## Building the circuit

The circuit is built as before after declaring the elementary models. The default case is solved to initialize all variables.

In [ ]:
from cosapp.systems import System
from cosapp.ports import Port
from cosapp.drivers import NonLinearSolver, RunSingleCase
import numpy as np


class Voltage(Port):
    def setup(self):
        self.add_variable('V', unit='V')

        
class Intensity(Port):
    def setup(self):
        self.add_variable('I', unit='A')
        
        
class Resistor(System):
    
    def setup(self, R = 1.):
        self.add_input(Voltage, 'V_in')
        self.add_input(Voltage, 'V_out')
        self.add_output(Intensity, 'I')
        
        self.add_inward('R', R, unit='ohm', desc='Resistance in Ohms')
        self.add_outward('deltaV', unit='V')  # Not mandatory; could be local to compute method
        
    def compute(self):
        self.deltaV = self.V_in.V - self.V_out.V
        self.I.I = self.deltaV / self.R        

        
class Node(System):
    
    def setup(self, n_in=1, n_out=1):
        self.add_property('n_in', int(n_in))
        self.add_property('n_out', int(n_out))

        if min(self.n_in, self.n_out) < 1:
            raise ValueError("Node needs at least one incoming and one outgoing current")

        for i in range(self.n_in):
            self.add_input(Intensity, f"I_in{i}")
        for i in range(self.n_out):
            self.add_input(Intensity, f"I_out{i}")
        
        self.add_inward('V')
        self.add_unknown('V')  # Iterative variable
        
        self.add_outward('sum_I_in', 0., desc='Sum of all incoming currents')
        self.add_outward('sum_I_out', 0., desc='Sum of all outgoing currents')
        
        self.add_equation('sum_I_in == sum_I_out', name='V')
        
    def compute(self):
        self.sum_I_in = sum(self[f"I_in{i}.I"] for i in range(self.n_in))
        self.sum_I_out = sum(self[f"I_out{i}.I"] for i in range(self.n_out))


class Source(System):
    
    def setup(self, I=0.1):
        self.add_inward('I', I)
        self.add_output(Intensity, 'I_out', {'I': I})
    
    def compute(self):
        self.I_out.I = self.I


class Ground(System):
    
    def setup(self, V=0.0):
        self.add_inward('V', V)
        self.add_output(Voltage, 'V_out', {'V': V})
    
    def compute(self):
        self.V_out.V = self.V
        

class Circuit(System):
    
    def setup(self):
        n1 = self.add_child(Node('n1', n_in=1, n_out=2), pulling={'I_in0': 'I_in'})
        n2 = self.add_child(Node('n2'))
        
        R1 = self.add_child(Resistor('R1', R=1000.), pulling={'V_out': 'Vg'})
        R2 = self.add_child(Resistor('R2', R=500.))
        R3 = self.add_child(Resistor('R3', R=250.), pulling={'V_out': 'Vg'})
        
        self.connect(R1.V_in, n1.inwards, 'V')
        self.connect(R2.V_in, n1.inwards, 'V')
        self.connect(R1.I, n1.I_out0)
        self.connect(R2.I, n1.I_out1)
        
        self.connect(R2.V_out, n2.inwards, 'V')
        self.connect(R3.V_in, n2.inwards, 'V')
        self.connect(R2.I, n2.I_in0)
        self.connect(R3.I, n2.I_out0)


p = System('model')
p.add_child(Source('source', I=0.1))
p.add_child(Ground('ground', V=0.))
p.add_child(Circuit('circuit'))

p.connect(p.source.I_out, p.circuit.I_in)
p.connect(p.ground.V_out, p.circuit.Vg)

solve = p.add_driver(NonLinearSolver('solve', verbose=True))
p.run_drivers()

# Printing some results
print('Resistances')
print(p.circuit.R1.R, p.circuit.R2.R, p.circuit.R3.R)
print('Current')
print(p.circuit.R1.I.I, p.circuit.R2.I.I, p.circuit.R3.I.I)
print('Voltage')
print(p.circuit.n1.V, p.circuit.n2.V)

## Defining the design case

After removing all previous drivers, the design case can be defined. First, a numerical solver is added - `NonLinearSolver`. Then for each design point a driver `RunSingleCase` is added as child to the numerical solver.

Setting values by a `RunSingleCase` is done through the method `set_values()`, passing a dictionary mapping a *variable name* to a *value*. In order to define the design variable and the associated equation on the operating point, methods `add_unknown()` and `add_equation()` of attribute `design` are called. An unknown is simply defined by its name; a design equation is defined by a string of the kind `"lhs == rhs"`, where `lhs` and `rhs` denote the left- and right-hand sides. Each side of the equation may be a constant or an evaluable expression, as in `"x - cos(sub.y) == z**2"`, or  `"x == 1.5"`. An exception will be raised if both sides are trivially constant, as in `"pi / 2 == 0"`.

In [ ]:
from cosapp.drivers import NonLinearMethods
from cosapp.recorders import DataFrameRecorder

p.drivers.clear()  # Clear all previously defined drivers
design = p.add_driver(NonLinearSolver('design', factor=0.2, tol=1e-6))  # Add numerical solver
design.add_recorder(DataFrameRecorder(includes=['*n?.V', '*R', 'source.I'], excludes='*R3*'))

# Add driver to set boundary conditions on point 1
point1 = design.add_child(RunSingleCase('pt1'))
# Same as previous for a second point
point2 = design.add_child(RunSingleCase('pt2'))

point1.set_values({
    'source.I': 0.08, 
    'ground.V': 0,
})
point1.design.add_unknown('circuit.R2.R').add_equation('circuit.n2.V == 8')

point2.set_values({
    'source.I': 0.15,
    'ground.V': 0,
})
point2.design.add_unknown('circuit.R1.R').add_equation('circuit.n1.V == 50')

p.run_drivers()
design.recorder.export_data()

## Validating the resolution

In [ ]:
from IPython.display import display

display(p.circuit.R1.inwards)
display(p.circuit.R2.inwards)

p.drivers.clear()
p.add_driver(NonLinearSolver('solver'))

p.source.I = 0.08
p.ground.V = 0
p.run_drivers()
print('n2.V =', p.circuit.n2.V)
assert abs(p.circuit.n2.V / 8 - 1) < 1e-4

p.source.I = 0.15
p.ground.V = 0
p.run_drivers()
print('n1.V =', p.circuit.n1.V)
assert abs(p.circuit.n1.V / 50 - 1) < 1e-4